In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, y_train = tf.cast(x_train, tf.float32), tf.cast(y_train, tf.int32)
x_test, y_test = tf.cast(x_test, tf.float32), tf.cast(y_test, tf.int32)

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train[..., tf.newaxis], x_test[..., tf.newaxis]

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
w_1 = tf.compat.v1.get_variable('w_1', shape = (3, 3, 1, 32), dtype = tf.float32, initializer = tf.initializers.glorot_uniform)

In [5]:
def MyModel(x):
    with tf.compat.v1.variable_scope("MyModel", reuse = tf.compat.v1.AUTO_REUSE):
        c_1 = tf.compat.v2.nn.conv2d(x, filters = w_1, strides = (1, 1), padding = "VALID")
        a_1 = tf.compat.v2.nn.relu(c_1)
        f_1 = tf.compat.v1.layers.flatten(a_1)
        d_1 = tf.compat.v1.layers.dense(f_1, units = 128, activation = tf.compat.v2.nn.relu)
        d_2 = tf.compat.v1.layers.dense(d_1, units = 10, activation = tf.compat.v2.nn.softmax)
    return d_2

In [6]:
loss_object = tf.compat.v2.losses.SparseCategoricalCrossentropy()

optimizer = tf.compat.v2.optimizers.Adam()

In [7]:
train_loss = tf.compat.v2.metrics.Mean(name='train_loss')
train_accuracy = tf.compat.v2.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.compat.v2.metrics.Mean(name='test_loss')
test_accuracy = tf.compat.v2.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = MyModel(images)
        loss = loss_object(labels, predictions)
    gradient = tape.gradient(loss, tf.compat.v1.trainable_variables())
    optimizer.apply_gradients(zip(gradient, tf.compat.v1.trainable_variables()))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [9]:
def test_step(images, labels):
    predictions = MyModel(images)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

In [10]:
EPOCHS = 3

for epoch in range(EPOCHS):
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for images, labels in test_ds:
        test_step(images, labels)
        
    tp = 'Epoch {:d}->Train Loss: {:.3f}, Train Accuracy: {:.3f}; Test Loss: {:.3f}, Test Accuracy: {:.3f}'

    print(tp.format(epoch+1, train_loss.result(), train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))
    
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()

W1117 10:41:07.203452 13376 deprecation.py:323] From <ipython-input-5-85ae7a69ee1c>:5: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1117 10:41:07.216453 13376 deprecation.py:323] From C:\Users\krishna rohit\Anaconda3\envs\machine-learning\lib\site-packages\tensorflow_core\python\layers\core.py:332: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W1117 10:41:07.226459 13376 deprecation.py:323] From <ipython-input-5-85ae7a69ee1c>:6: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.


Epoch 1->Train Loss: 2.304, Train Accuracy: 10.085; Test Loss: 2.305, Test Accuracy: 10.190
Epoch 2->Train Loss: 2.304, Train Accuracy: 10.040; Test Loss: 2.304, Test Accuracy: 10.670
Epoch 3->Train Loss: 2.305, Train Accuracy: 10.077; Test Loss: 2.305, Test Accuracy: 9.360
